# Flatten gridded data for trend fitting

In [54]:
# import sys; sys.path.append('/rds/general/user/cb2714/home/wwa'); from wwa import *
import xarray as xr, regionmask

# make a new directory to store the data & results
! mkdir -p flattened; mkdir -p flattened-res

In [55]:
# load daily data
da = xr.open_dataset("data/pr_era5_-10_5_30_45.nc").tp.rename("pr")

# mask land only
rm = regionmask.defined_regions.natural_earth_v5_0_0.land_10.mask(da.lon, da.lat)
da = da.where(rm == 0, drop = True)

## Edit the cells below to reflect your data & event definition

In [56]:
# extract the event
ev = da.sel(time = "2026-01-26")

In [57]:
# use your event definition to get an annual time series at each grid cell
da_ann = da.sel(time = da.time.dt.month.isin([10,11,12,1,2,3])).resample(time = "YE-JUL").max()

# set a file description that will be used to save all the data
fnm_root = "rx1day-era5-260126"

## Run the cell below to flatten the data and save ready for processing

In [58]:
# merge event with  main time series (put event first so it's easier to identify)
da_ann = xr.concat([ev, da_ann], "time")

In [59]:
# relabel dates as years
da_ann = da_ann.assign_coords(time = da_ann.time.dt.year).rename(time = "year")

# save the map for easier reconstruction later
da_ann.mean("year").to_netcdf("map-tmplt_"+fnm_root+".nc")

# flatten & convert to data.frame
df = da_ann.stack(xy = ["lat", "lon"]).dropna(dim = "xy", how = "all").to_pandas()

In [60]:
# save data.frame as .csv (split into chunks if really large)
ncols = 2550
if df.shape[1] > ncols:
    for i in range(int(np.ceil(df.shape[1] / ncols))):
        df.iloc[:,slice(i*ncols,(i+1)*ncols)].to_csv("flattened/"+fnm_root+"-flattened_"+str(i+1).rjust(2,"0")+".csv")
else:
    df.to_csv("flattened/"+fnm_root+"-flattened.csv")

**Now open the next workbook (using R) to fit the statistical model over each grid cell**